# Mini-Data Set Preparation

After the Kaggle Script "Making a mini-data set" is run (FYI, it takes about 2 minutes to run) to reduce the size of the data to 40,000 instances, run this script to organize data into a single dataframe. 

Run this with the 8 csv files produced by the Kaggle Script in the same directory. 

Note: This is a Python3 script because that is what Kaggle uses. 

In [1]:
import pandas as pd
import numpy as np
import copy
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression

In [ ]:
clicks_train = pd.read_csv("../input/clicks_train.csv")
clicks_train = clicks_train.sample(10000, random_state = 0)
# random sample of the clicks (which displays were clicked)

promoted_content = pd.read_csv("../input/promoted_content.csv")
promoted = promoted_content[promoted_content["ad_id"].isin(mini_clicks_train["ad_id"])]
# ads that were in the random sample of clicks

doc_cats = pd.read_csv("../input/documents_categories.csv")
doc_cats = doc_cats[doc_cats["document_id"].isin(mini_promoted["document_id"])]

doc_ents = pd.read_csv("../input/documents_entities.csv")
doc_ents = doc_ents[doc_ents["document_id"].isin(mini_promoted["document_id"])]

doc_meta = pd.read_csv("../input/documents_meta.csv")
doc_meta = doc_meta[doc_meta["document_id"].isin(mini_promoted["document_id"])]

doc_topics = pd.read_csv("../input/documents_topics.csv")
doc_topics = doc_topics[doc_topics["document_id"].isin(mini_promoted["document_id"])]

events = pd.read_csv("../input/events.csv")
events = events[events["display_id"].isin(mini_clicks_train["display_id"])]

page_views = pd.read_csv("../input/page_views_sample.csv")
page_views = page_views[page_views["document_id"].isin(mini_promoted["document_id"])]


## Join clicks_train and events on display_id

In [3]:
#clicks_train and events have a 1:1 relationship
print(len(events["display_id"].unique()))
print(len(clicks_train["display_id"].unique()))

9994
9994


In [4]:
#the first column seems to be the old index, we don't need this
clicks_train = clicks_train.set_index('display_id')
del clicks_train["Unnamed: 0"]
clicks_train.head()

,ad_id,clicked
display_id,,
7194604,189575,1
10660899,281556,0
4510419,422667,0
2203285,11189,0
12632400,470488,0


In [5]:
del events["Unnamed: 0"]
events = events.set_index("display_id")
events.head()

,uuid,document_id,timestamp,platform,geo_location
display_id,,,,,
8112,715ddc85c78e39,1792396,559250,2,US>MI>563
10333,ec3841c3250fb6,1405372,716674,2,US>MO>616
11278,324475e5f191c8,1179111,784022,2,US>GA>524
13320,27221a85141108,735143,926961,2,US>CA>807
13905,9ea0907cee579e,1794259,971672,2,US>IL>602


In [6]:
data = clicks_train.join(events)
data.head()

,ad_id,clicked,uuid,document_id,timestamp,platform,geo_location
display_id,,,,,,,
8112,151682,1,715ddc85c78e39,1792396,559250,2,US>MI>563
10333,157598,0,ec3841c3250fb6,1405372,716674,2,US>MO>616
11278,103712,1,324475e5f191c8,1179111,784022,2,US>GA>524
13320,139944,1,27221a85141108,735143,926961,2,US>CA>807
13905,173393,0,9ea0907cee579e,1794259,971672,2,US>IL>602


## Promoted

In [7]:
len(promoted)

6042

In [8]:
#there is not a one-to-one relationship between document_id in promoted and the master data
#This is because the same ad is being shown in different documents I think
print(len(promoted["document_id"].unique()))
print(len(data["document_id"].unique()))

4230
7240


In [9]:
#there is a one-to-one relationship between ad_id in promoted and the master data
print(len(promoted["ad_id"].unique())) #each add can appear more than once
print(len(data["ad_id"].unique()))

6042
6042


In [10]:
promoted.head()
del promoted["Unnamed: 0"]
del promoted['document_id'] #I think all we want from here is the link between ad_id and campaign id
promoted.head()

,ad_id,campaign_id,advertiser_id
0,489,81,116
1,844,37,63
2,971,28,45
3,1117,205,134
4,1126,209,295


## Joining Info about each ad

I make a dictionary of the advertiser and campaign id for each ad_id, map that dictionary to the ad id to make the advertizer and campain columns

In [11]:
data.head()

,ad_id,clicked,uuid,document_id,timestamp,platform,geo_location
display_id,,,,,,,
8112,151682,1,715ddc85c78e39,1792396,559250,2,US>MI>563
10333,157598,0,ec3841c3250fb6,1405372,716674,2,US>MO>616
11278,103712,1,324475e5f191c8,1179111,784022,2,US>GA>524
13320,139944,1,27221a85141108,735143,926961,2,US>CA>807
13905,173393,0,9ea0907cee579e,1794259,971672,2,US>IL>602


In [12]:
print(len(data))
print(len(data["ad_id"].unique())) #adds appear on average slightly more than twice in our minidata set

10000
6042


In [13]:
#make dictionaries to look up advertizer id and campaign id for each ad_id
advertiser_dict = dict(zip(promoted.ad_id, promoted.advertiser_id))
campaign_dict = dict(zip(promoted.ad_id, promoted.campaign_id))


In [14]:
data["campaign_id"] = data["ad_id"].map(campaign_dict)
data["advertiser_id"] = data["ad_id"].map(advertiser_dict)
data.head()

,ad_id,clicked,uuid,document_id,timestamp,platform,geo_location,campaign_id,advertiser_id
display_id,,,,,,,,,
8112,151682,1,715ddc85c78e39,1792396,559250,2,US>MI>563,19284,3283
10333,157598,0,ec3841c3250fb6,1405372,716674,2,US>MO>616,18022,2312
11278,103712,1,324475e5f191c8,1179111,784022,2,US>GA>524,5081,1759
13320,139944,1,27221a85141108,735143,926961,2,US>CA>807,17929,1782
13905,173393,0,9ea0907cee579e,1794259,971672,2,US>IL>602,21143,2876


In [15]:
print(len(data))
print(len(data["ad_id"].unique())) #adds appear on average slightly more than twice in our minidata set

10000
6042


## Working with Page Views

Add count of page views to every document in data as a feature, could tell us something about likelihood of ads being clicked

In [61]:
print(len(page_views))
print(len(page_views['document_id'].unique()))
# when we filter our data like this, we only get 23 documents from page_views_sample.. we need the full dataset


36071
23


## Importing Document Information

I'm super stuck on why all the document ids that appear in our data arent in the files with more information about each documents.

In [16]:
#Why aren't there the same number of unique documents in each of these
print(len(data["document_id"].unique()))
print(len(doc_cats["document_id"].unique()))
print(len(doc_ents["document_id"].unique()))
print(len(doc_meta["document_id"].unique()))
print(len(doc_topics["document_id"].unique()))

7240
4223
3540
4230
4154


In [17]:
#each document has multiple possible entities, categories, topics with different confidence level. 
#maybe we should just for now keep the most likely entity, topic and category? 
doc_ents.head()

,Unnamed: 0,document_id,entity_id,confidence_level
0,72655,1807706,11d5279de0b36011773ee50f75f9b43c,0.495961
1,72656,1807706,358c9e8a2ff80f858f6deb063e0bcf8f,0.337521
2,72657,1807706,b2a032747f34ec8488b2be2ebe180d9e,0.281354
3,72673,1316839,e8493cde7d253c7def5b030a2c94357d,0.977918
4,72674,1316839,c5039d9dda9bccb8ed150d8affedd306,0.375150


In [18]:
doc_cats.head()

,Unnamed: 0,document_id,category_id,confidence_level
0,51976,1807706,1209,0.92
1,51977,1807706,1205,0.07
2,51990,1316839,2003,0.92
3,51991,1316839,1211,0.07
4,51994,479029,1205,0.92


In [19]:
data.head()

,ad_id,clicked,uuid,document_id,timestamp,platform,geo_location,campaign_id,advertiser_id
display_id,,,,,,,,,
8112,151682,1,715ddc85c78e39,1792396,559250,2,US>MI>563,19284,3283
10333,157598,0,ec3841c3250fb6,1405372,716674,2,US>MO>616,18022,2312
11278,103712,1,324475e5f191c8,1179111,784022,2,US>GA>524,5081,1759
13320,139944,1,27221a85141108,735143,926961,2,US>CA>807,17929,1782
13905,173393,0,9ea0907cee579e,1794259,971672,2,US>IL>602,21143,2876


In [20]:
data["geo_location"] = data["geo_location"].apply(str)

In [21]:
data["country"] = [x[:2] for x in data["geo_location"]]
data["state"] = [x[3:5] for x in data["geo_location"]]
data["loc_num"]= [x[6:] for x in data["geo_location"]]
data["loc_num"] = data["loc_num"].map(str)
data.head()

,ad_id,clicked,uuid,document_id,timestamp,platform,geo_location,campaign_id,advertiser_id,country,state,loc_num
display_id,,,,,,,,,,,,
8112,151682,1,715ddc85c78e39,1792396,559250,2,US>MI>563,19284,3283,US,MI,563
10333,157598,0,ec3841c3250fb6,1405372,716674,2,US>MO>616,18022,2312,US,MO,616
11278,103712,1,324475e5f191c8,1179111,784022,2,US>GA>524,5081,1759,US,GA,524
13320,139944,1,27221a85141108,735143,926961,2,US>CA>807,17929,1782,US,CA,807
13905,173393,0,9ea0907cee579e,1794259,971672,2,US>IL>602,21143,2876,US,IL,602


In [22]:
list_to_binarize = ["advertiser_id", "campaign_id", "platform", "country"]

for i in list_to_binarize:
    temp = pd.get_dummies(data[i], prefix = i)
    data = data.join(temp)
data.head()

,ad_id,clicked,uuid,document_id,timestamp,platform,geo_location,campaign_id,advertiser_id,country,...,country_TZ,country_UA,country_UG,country_US,country_UZ,country_VI,country_VN,country_ZA,country_ZM,country_ZW
display_id,,,,,,,,,,,,,,,,,,,,,
8112,151682,1,715ddc85c78e39,1792396,559250,2,US>MI>563,19284,3283,US,...,0,0,0,1,0,0,0,0,0,0
10333,157598,0,ec3841c3250fb6,1405372,716674,2,US>MO>616,18022,2312,US,...,0,0,0,1,0,0,0,0,0,0
11278,103712,1,324475e5f191c8,1179111,784022,2,US>GA>524,5081,1759,US,...,0,0,0,1,0,0,0,0,0,0
13320,139944,1,27221a85141108,735143,926961,2,US>CA>807,17929,1782,US,...,0,0,0,1,0,0,0,0,0,0
13905,173393,0,9ea0907cee579e,1794259,971672,2,US>IL>602,21143,2876,US,...,0,0,0,1,0,0,0,0,0,0


In [34]:
clean = copy.deepcopy(data)
clean.head()

,ad_id,clicked,uuid,document_id,timestamp,platform,geo_location,campaign_id,advertiser_id,country,...,country_TZ,country_UA,country_UG,country_US,country_UZ,country_VI,country_VN,country_ZA,country_ZM,country_ZW
display_id,,,,,,,,,,,,,,,,,,,,,
8112,151682,1,715ddc85c78e39,1792396,559250,2,US>MI>563,19284,3283,US,...,0,0,0,1,0,0,0,0,0,0
10333,157598,0,ec3841c3250fb6,1405372,716674,2,US>MO>616,18022,2312,US,...,0,0,0,1,0,0,0,0,0,0
11278,103712,1,324475e5f191c8,1179111,784022,2,US>GA>524,5081,1759,US,...,0,0,0,1,0,0,0,0,0,0
13320,139944,1,27221a85141108,735143,926961,2,US>CA>807,17929,1782,US,...,0,0,0,1,0,0,0,0,0,0
13905,173393,0,9ea0907cee579e,1794259,971672,2,US>IL>602,21143,2876,US,...,0,0,0,1,0,0,0,0,0,0


In [35]:
len(data["ad_id"].unique())

6042

In [36]:
len(clean['document_id'].unique())

7240

In [37]:
#drop the non-cleaned up data for now
del clean['country'], clean['state'], clean["ad_id"], clean["uuid"], clean["document_id"],clean["timestamp"], clean["platform"], clean["geo_location"], clean['campaign_id'], clean["advertiser_id"]

In [39]:
len(clean.columns)
clean.head()

,clicked,loc_num,advertiser_id_4,advertiser_id_5,advertiser_id_8,advertiser_id_9,advertiser_id_10,advertiser_id_12,advertiser_id_13,advertiser_id_14,...,country_TZ,country_UA,country_UG,country_US,country_UZ,country_VI,country_VN,country_ZA,country_ZM,country_ZW
display_id,,,,,,,,,,,,,,,,,,,,,
8112,1,563,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
10333,0,616,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
11278,1,524,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
13320,1,807,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
13905,0,602,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


## Make training and test sets

In [45]:
labels = clean['clicked']
labels = labels.values.reshape(-1,1) # check this please! my python is 3.5 and told me to use values.reshape
del clean['clicked']

In [55]:
train_data = clean[:round(.7*len(clean))]
train_labels = labels[:round(.7*len(clean))]
test_data = clean[round(.7*len(clean)):]
test_labels = labels[round(.7*len(clean)):]

In [56]:
lr = LogisticRegression()
lr.fit(train_data, train_labels)
lr.predict(test_data, test_labels)

# Logistic Regression is not the best model because we have to binarize our features
# to make them numeric. This leads to >5000 columns which is unruly and not optimal.

ValueError: could not convert string to float: 